In [ ]:
 [code]

# Problem 4 Image Classification

In [ ]:
import sys
import gc
import random
import glob
import math
import itertools
import numpy as np
import pandas as pd
import albumentations
import cv2
import time
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
import IPython
from IPython.display import display
import pathlib
# Import all libraries I can think of
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
import tensorflow_datasets as tfds

In [ ]:
sys.path.append('../input/timmmaster')
import timm

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
print(device)

## *NO GPU for us* ##

## Disclaimer: Since I donot have a GPU, I will train a very simple classifier.
## The first line below the objective gives it away that this is likely a kaggle project
## My knowledge on Image classification rather limited so work below is heavily inspired by nitibbgg

In [ ]:
path = '/kaggle/input/paddy-disease-classification'

In [ ]:
train = pd.read_csv(path+'/train.csv')
train.shape

In [ ]:
train.head()

Taking a look at the an sample image corresponding to the 1st row above.

In [ ]:
trainimg_path = path+'/train_images'
img = Image.open(trainimg_path+'/bacterial_leaf_blight/100330.jpg')
# Get the image size
img_size = img.size
print("The image size is: {}".format(img_size))
img

In [ ]:
# Train test split of 25%
train = tf.keras.utils.image_dataset_from_directory(
  trainimg_path,
  validation_split=0.25,
  subset="training",
  seed=123,
  image_size=(120, 160),
  batch_size=16)

In [ ]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.25,
  subset="validation",
  seed=123,
  image_size=(120, 160),
  batch_size=16)

In [ ]:
class_names = train.class_names
plt.figure(figsize=(15, 15))
for images, labels in train.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")

This section to set params for th cv model is heavily inspired by nitibbgg

In [ ]:
img_height = 120
img_width = 160
batch_size=16
### Section heavily inspired by 
data_augmentation = tf.keras.Sequential(
    [
        tf.keras.layers.Normalization(),
        tf.keras.layers.Resizing(img_height, img_width),
        tf.keras.layers.Rescaling(1./4),  
        tf.keras.layers.RandomFlip("horizontal"),
        tf.keras.layers.RandomRotation(factor=0.02),
        tf.keras.layers.RandomZoom(
            height_factor=0.4, width_factor=0.4
        ),
    ],
    name="data_augmentation",
)
train = train.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
early_stopping_cb = tf.keras.callbacks.EarlyStopping(
    patience=5, restore_best_weights=True)
num_classes = len(class_names)
model = tf.keras.Sequential([
  data_augmentation,
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(num_classes)
])
model.build((None,img_height,img_width,3))
model.summary()

In [ ]:
model.compile(
  optimizer='adam',
  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

Running the model with epoch 5 in the interest of time and **CPU usage > 300% !!**

In [ ]:
model.fit(
  train,
  validation_data=val_ds,
  epochs=5,
  callbacks=[early_stopping_cb],
)

In [ ]:
test_path = pathlib.Path(path+'/test_images/')
test_ds = tf.keras.utils.image_dataset_from_directory(
    test_path,
    label_mode=None,
    shuffle=False,
    image_size=(img_height, img_width),
    batch_size=batch_size)

# Predicting the values on

In [ ]:
y_pred =  model.predict(test_ds)
y_pred_classes = y_pred.argmax(axis=1)

In [ ]:
np.unique(y_pred_classes)